In [ ]:
%matplotlib widget

import os
import pathlib
import pprint

from Visualizer.DataSet import DataSet
from Visualizer.Monitoring.Persistence.ProfilerSession import ProfilerSession
from Visualizer.Plotting.Plot import Plot

# [Analyzer] Single Profiler Session

This notebook can visualize a specific profiler session.

## Configuration

Configure the project settings below.
The default configuration should be okay.
Make sure that you have run at least one profiler session with the EnergyManager and have generated a database, otherwise this notebook will fail.

In [ ]:
source_directory = pathlib.Path().parent.absolute()
project_directory = f"{source_directory}/.."
root_directory = f"{project_directory}/.."
resources_directory = f"{root_directory}/Resources"
database = f"{resources_directory}/Database.sqlite"

# Load all profiler sessions
complete_data_set = DataSet(ProfilerSession.load_all(database))

### Profiler Sessions Overview

Below all profiler sessions are shown.

In [ ]:
# Visualize the profiler session data
overview_plot = complete_data_set.table_plot
overview_plot.maximum_rows = 500
overview_plot.plot()

## Analysis

Define the ID of the test you wish to visualize below.

In [ ]:
# profiler_session_id = complete_data_set.data[-1].id
profiler_session_id = 85
data_set = DataSet([profiler_session for profiler_session in complete_data_set.data if profiler_session.id == profiler_session_id])

### Summary

These are the specifications of the system with which the profiler session was performed.

In [ ]:
pprint.pprint(data_set.data[0].summary)

### Profile

This is the profile with which the profiler session was performed.

In [ ]:
pprint.pprint(data_set.data[0].profile)

The graph below shows the clock rate over time.

In [ ]:
clock_rate_timeseries_plot = data_set.data[0].clock_rate_timeseries_plot(plot_limits=False, use_ear=False)
clock_rate_timeseries_plot.plot_series = { key: clock_rate_timeseries_plot.plot_series[key] for key in clock_rate_timeseries_plot.plot_series if (key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0")}
clock_rate_timeseries_plot.plot()

### Correlations

The graphs below shows the Pearson correlation heatmap for each performance variable.

In [ ]:
#data_set.data[0].monitor_data_correlations_plot.plot()

### Energy Consumption

The graphs below shows the total energy consumption over time.

In [ ]:
#### # Plot in Joules
energy_consumption_plot_j = data_set.data[0].energy_consumption_timeseries_plot(use_ear=False)
energy_consumption_plot_j.plot_series = { key: energy_consumption_plot_j.plot_series[key] for key in energy_consumption_plot_j.plot_series if (key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
energy_consumption_plot_j.plot()

# Plot in Watt Hours
# energy_consumption_plot_wh = data_set.data[0].energy_consumption_timeseries_plot(unit_string="Wh", modifier=Plot.j_to_wh, use_ear=False)
# energy_consumption_plot_wh.plot_series = { key: energy_consumption_plot_wh.plot_series[key] for key in energy_consumption_plot_wh.plot_series if (key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
# energy_consumption_plot_wh.plot()

### Events

The graph below shows the events over time.

In [ ]:
# for events_plot in data_set.data[0].events_event_plot()[:1]:
#     events_plot.plot()

### Fan Speed

The graphs below shows the fan speed over time.

In [ ]:
data_set.data[0].fan_speed_timeseries_plot.plot()

### Instructions Per Cycle

The graph below shows the utilization rates over time.

In [ ]:
data_set.data[0].instructions_per_cycle_timeseries_plot.plot()

### Memory Consumption

The graphs below shows the memory consumption over time.

In [ ]:
data_set.data[0].memory_consumption_timeseries_plot(plot_sizes=False).plot()

### Power Consumption

The graph below shows the power consumption over time.

In [ ]:
power_consumption_timeseries_plot = data_set.data[0].power_consumption_timeseries_plot(plot_limits=False, use_ear=False)
#power_consumption_timeseries_plot.plot_series = { key: power_consumption_timeseries_plot.plot_series[key] for key in power_consumption_timeseries_plot.plot_series if (key.startswith("CPU 1") or key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
power_consumption_timeseries_plot.plot_series = { key: power_consumption_timeseries_plot.plot_series[key] for key in power_consumption_timeseries_plot.plot_series if (key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
power_consumption_timeseries_plot.plot()

### Processes

The graphs below shows the amount of processes over time.

In [ ]:
data_set.data[0].processes_timeseries_plot.plot()

### States

The graph below shows the states of the EnergyMonitor over time.

In [ ]:
data_set.data[0].states_event_plot().plot()

In [ ]:
total = 0
idle = 0
gpu_idle = 0
cpu_idle = 0
busy = 0
cpu_busy_wait = 0

for (state, begin, end) in data_set.data[0].states:
    diff = (end - begin).total_seconds()
    total += diff
    if (state == "IDLE"):
        idle += diff
    elif (state == "GPU_IDLE"):
        gpu_idle += diff
    elif (state == "CPU_IDLE"):
        cpu_idle += diff
    elif (state == "BUSY"):
        busy += diff
    elif (state == "CPU_BUSY_WAIT"):
        cpu_busy_wait += diff
    
n = 1
print("IDLE:", round(idle/total*100, n), "%\nCPU_IDLE:", round(cpu_idle/total*100, n), "%\nGPU_IDLE:", round(gpu_idle/total*100, n), "%\nCPU_BUSY_WAIT:", round(cpu_busy_wait/total*100, n), "%\nBUSY:", round(busy/total*100, n), "%")

### Temperature

The graphs below shows the temperature over time.

In [ ]:
temperature_timeseries_plot = data_set.data[0].temperature_timeseries_plot
temperature_timeseries_plot.plot_series = { key: temperature_timeseries_plot.plot_series[key] for key in temperature_timeseries_plot.plot_series if (key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
temperature_timeseries_plot.plot()

### Timespan

The graphs below shows the time spent on different types of operations over time.

In [ ]:
timespan_timeseries_plot = data_set.data[0].timespan_timeseries_plot(use_ear=False)
timespan_timeseries_plot.plot_series = { key: timespan_timeseries_plot.plot_series[key] for key in timespan_timeseries_plot.plot_series if (key.startswith("Runtime") or key.startswith("EAR") or key.startswith("CPU 0 Core 0"))}
timespan_timeseries_plot.plot()

### Utilization Rates

The graph below shows the utilization rates over time.

In [ ]:
utilization_rate_timeseries_plot = data_set.data[0].utilization_rate_timeseries_plot
utilization_rate_timeseries_plot.plot_series = { key: utilization_rate_timeseries_plot.plot_series[key] for key in utilization_rate_timeseries_plot.plot_series if (key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
# utilization_rate_timeseries_plot.plot_series = { key: utilization_rate_timeseries_plot.plot_series[key] for key in utilization_rate_timeseries_plot.plot_series if (key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 2")) or (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
utilization_rate_timeseries_plot.plot()

# 